In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
train=pd.read_csv('trainair.csv')

In [4]:
test=pd.read_csv('testair.csv')

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33750 entries, 0 to 33749
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date_time            33750 non-null  object 
 1   is_holiday           33750 non-null  object 
 2   humidity             33750 non-null  int64  
 3   wind_speed           33750 non-null  int64  
 4   wind_direction       33750 non-null  int64  
 5   visibility_in_miles  33750 non-null  int64  
 6   dew_point            33750 non-null  int64  
 7   temperature          33750 non-null  float64
 8   rain_p_h             33750 non-null  float64
 9   snow_p_h             33750 non-null  float64
 10  clouds_all           33750 non-null  int64  
 11  weather_type         33750 non-null  object 
 12  air_pollution_index  33750 non-null  int64  
 13  traffic_volume       33750 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 3.6+ MB


In [6]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [7]:
train['is_holiday']=le.fit_transform(train['is_holiday'])

In [8]:
train['weather_type']=le.fit_transform(train['weather_type'])

In [9]:
y=train.iloc[:,12:13]
X=train.iloc[:,[1,2,3,4,5,6,7,8,9,10,11,13]]

In [10]:
y.shape

(33750, 1)

In [11]:
X.shape

(33750, 12)

In [12]:
from sklearn.metrics import mean_absolute_error

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [15]:
def evaluate(model,train_features,train_labels,test_features,test_labels):
    train_pred=model.predict(train_features)
    test_pred=model.predict(test_features)
    train_error=mean_absolute_error(train_labels,train_pred)
    train_error=np.sqrt(train_error)
    test_error=mean_absolute_error(test_labels,test_pred)
    test_error=np.sqrt(test_error)
    print('Train Set Error: %0.2f' % train_error)
    print('Test Set Error: %0.2f' % test_error)

def display_scores(scores):
    print('Scores:', scores)

In [16]:
import xgboost as xgb

In [144]:
regressor=xgb.XGBRegressor(n_estimators=420,max_depth=2,learning_rate=0.0109)
regressor.fit(X_train,y_train)

[00:35:29] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.0109, max_delta_step=0,
             max_depth=2, min_child_weight=1, missing=None, n_estimators=420,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [145]:
regressor_evaluate=evaluate(regressor,X_train,y_train,X_test,y_test)

Train Set Error: 8.51
Test Set Error: 8.51


In [146]:
from sklearn.model_selection import RandomizedSearchCV

In [147]:
booster=['gbtree','gblinear']
base_score=[0.25,0.5,0.75,1]
n_estimators = [100, 200,300,350,400,450]
max_depth = [2, 3, 5]
booster=['gbtree','gblinear']
learning_rate=[0.01,0.02]
min_child_weight=[1,2,3,4]

In [148]:

hyperparameter_grid = {
    'n_estimators': n_estimators,
    'max_depth':max_depth,
    'learning_rate':learning_rate,
    'min_child_weight':min_child_weight,
    'booster':booster,
    'base_score':base_score
    }

In [151]:
random_cv = RandomizedSearchCV(estimator=regressor,
            param_distributions=hyperparameter_grid,
            cv=5, n_iter=50,
            scoring = 'neg_mean_absolute_error',n_jobs = 4,
            verbose = 5, 
            return_train_score = True,
            random_state=42)

In [152]:
random_cv.fit(X_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   21.8s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  2.7min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  5.3min
[Parallel(n_jobs=4)]: Done 250 out of 250 | elapsed:  8.4min finished


[01:48:23] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.0109,
                                          max_delta_step=0, max_depth=2,
                                          min_child_weight=1, missing=None,
                                          n_estimators=420, n_jobs=1,
                                          nthread=None, objective='reg:linear',
                                          random_state=0, reg_alp...
                   iid='deprecated', n_iter=50, n_jobs=4,
                   param_distributions={'base_score': [0.25, 0.5, 0.75, 1],
                                        'booster': ['g

In [153]:
random_cv.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=2, min_child_weight=2, missing=None, n_estimators=450,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [154]:
random_cv.best_estimator_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=2, min_child_weight=2, missing=None, n_estimators=450,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [1060]:
regressor=xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=2,
             importance_type='gain', learning_rate=0.0109, max_delta_step=0,
             max_depth=2, min_child_weight=2, missing=None, n_estimators=399,
             n_jobs=4, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=101,
             silent=None, subsample=1, verbosity=1)

In [1061]:
regressor.fit(X_train,y_train)

[05:13:04] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=2,
             importance_type='gain', learning_rate=0.0109, max_delta_step=0,
             max_depth=2, min_child_weight=2, missing=None, n_estimators=399,
             n_jobs=4, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=101,
             silent=None, subsample=1, verbosity=1)

In [1062]:
regressor_evaluate=evaluate(regressor,X_train,y_train,X_test,y_test)

Train Set Error: 8.51
Test Set Error: 8.51


In [1063]:
test['is_holiday']=le.fit_transform(test['is_holiday'])

In [1064]:
test['weather_type']=le.fit_transform(test['weather_type'])

In [1065]:
test.drop('date_time',axis=1)

,is_holiday,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,traffic_volume
0,7,63,1,27,4,4,285.15,0.0,0,90,6,646
1,7,63,1,27,4,4,285.15,0.0,0,90,5,646
2,7,56,1,0,1,1,285.15,0.0,0,90,2,646
3,7,56,1,351,2,2,284.79,0.0,0,90,6,353
4,7,56,1,351,1,1,284.79,0.0,0,90,5,353
...,...,...,...,...,...,...,...,...,...,...,...,...
14449,7,21,4,345,4,4,283.45,0.0,0,75,1,3543
14450,7,95,8,280,6,6,282.76,0.0,0,90,1,2781
14451,7,63,4,238,1,1,282.73,0.0,0,90,9,2159
14452,7,57,8,268,7,7,282.09,0.0,0,90,1,1450


In [1066]:
lgb_gfdct = regressor.fit(train.drop(['air_pollution_index'], axis=1), train['air_pollution_index'])

[05:13:10] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [1067]:
lgb1_pred=lgb_gfdct.predict(test.drop('date_time',axis=1))

In [1068]:
sub=[]
for i in range (0,14454):
    sub.append(lgb1_pred[i])

In [1069]:
submission = pd.DataFrame()

In [1070]:
submission['date_time']=test['date_time']
submission['air_pollution_index']=lgb1_pred

In [1071]:
submission

,date_time,air_pollution_index
0,2017-05-18 00:00:00,151.604462
1,2017-05-18 00:00:00,151.604462
2,2017-05-18 00:00:00,151.604462
3,2017-05-18 01:00:00,155.353485
4,2017-05-18 01:00:00,155.353485
...,...,...
14449,2018-09-30 19:00:00,152.235367
14450,2018-09-30 20:00:00,152.737930
14451,2018-09-30 21:00:00,152.743942
14452,2018-09-30 22:00:00,152.516617


In [1072]:
submission.to_csv('sub_air.csv', index=False)